In [7]:
"""
Aashish Adhikari
"""


from __future__ import division
from __future__ import print_function

import sys
try:
   import _pickle as pickle
except:
   import pickle

import numpy as np



# This is a class for a LinearTransform layer which takes an input 
# weight matrix W and computes W x as the forward step



class LinearTransform(object):

    def __init__(self, W, b):
        self.weights = W
        self.bias = b
    def forward(self, x):
        print()
    def backward(
        self, 
        grad_output, 
        learning_rate=0.0, 
        momentum=0.0, 
        l2_penalty=0.0,
    ):
        print()
	# DEFINE backward function
# ADD other operations in LinearTransform if needed



# This is a class for a ReLU layer max(x,0)



class ReLU(object):

    def forward(self, x):
	# DEFINE forward function
        print()
    def backward(
        self, 
        grad_output, 
        learning_rate=0.0, 
        momentum=0.0, 
        l2_penalty=0.0,
    ):
        print()
    # DEFINE backward function
# ADD other operations in ReLU if needed



# This is a class for a sigmoid layer followed by a cross entropy layer, the reason 
# this is put into a single layer is because it has a simple gradient form



class SigmoidCrossEntropy(object):
    def forward(self, x):
        print()
        
    def backward(self,grad_output,learning_rate=0.0,momentum=0.0,l2_penalty=0.0):
        print()




# This is a class for the Multilayer perceptron




class MLP(object):

    def __init__(self, input_dims, hidden_units):
    # INSERT CODE for initializing the network
#         self.input_dims = input_dims
#         self.hidden_units = hidden_units
        self.first_Layer_Weights = np.random.rand(input_dims, hidden_units)
        print(self.first_Layer_Weights.shape)

    def train(self,x_batch,y_batch,learning_rate,momentum,l2_penalty):
        
        for sample,label in (x_batch, y_batch):
            #yeha bata linear transform ma pathai
            
            
        #one by one each example lai agadi pathaune 10 ota palai palo ayepaxi tiotal cumulative ma kaam garne yeha
        
        print()
        
        ######Yeha k garne bhar aaba
        
        
	# INSERT CODE for training the network

    
    
    
    
    def evaluate(self, x, y):
        print()
	# INSERT CODE for testing the network
# ADD other operations and data entries in MLP if needed

if __name__ == '__main__':
    if sys.version_info[0] < 3:
        print("system version is less than 3")
        #train_x, train_y, test_x, test_y = 
        data = pickle.load(open('dataset_folder/cifar_2class_py2.p', 'rb'))
        
    else:
	    #train_x, train_y, test_x, test_y 
        data = pickle.load(open('dataset_folder/cifar_2class_py2.p', 'rb'), encoding='bytes')
    #print(data)
    
    #print(data[b'test_data'])
    train_x = data[b'train_data']
    train_y = data[b'train_labels']
    test_x = data[b'test_data']
    test_y = data[b'test_labels']
    
#     print(test_y)
	
    num_examples, input_dims = train_x.shape
	# INSERT YOUR CODE HERE
    #ask the user about the the number of hidden nodes hs/ she wants
    
    #inp = input("How many hidden nodes do you want?\n")
    #num_of_hidden_nodes = int(inp)
    num_of_hidden_nodes = 2
    print("dimension of each example is ",input_dims)
    
    
	
    
    
    # YOU CAN CHANGE num_epochs AND num_batches TO YOUR DESIRED VALUES
    num_epochs = 10
    num_batches = 1000
    learning_rate = [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]
    inertia_of_momentum = [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]
    l2_penalty_factor = [0.0000001,0.0000003,0.000001,0.000003,0.00001,0.00003,0.0001,0.0003,0.001,0.003,0.01,0.03,1,3,10]
    print("Choose the corresponding index number for the learning rate you want to use")
    lr = int(input("/[0.001,0.003,0.01,0.03,0.1,0.3,1,3,10/]"))
    print("Choose the corresponding index number for the inertia of momentum you want to use")
    iner = int(input("/[0.001,0.003,0.01,0.03,0.1,0.3,1,3,10/]"))
    print("Choose the corresponding index number for the L2 penalty factor you want to use")
    penalty = int(input("/[0.0000001,0.0000003,0.000001,0.000003,0.00001,0.00003,0.0001,0.0003,0.001,0.003,0.01,0.03,1,3,10/]"))
    mlp = MLP(input_dims, num_of_hidden_nodes)
    
    for epoch in range(num_epochs):

	# INSERT YOUR CODE FOR EACH EPOCH HERE

        for b in range(num_batches):
            total_loss = 0.0
            print("num of examples num of batches ",num_examples, " ",num_batches)
            
            batch_start = int( (num_examples / num_batches) * b)
            batch_end = int((num_examples / num_batches)*(b +1) - 1)
            print("batch start and end",batch_start," ",batch_end)
            print("Chosen learning rate, inertia of momentum and l2 penalty factor are")
            print(learning_rate[lr], inertia_of_momentum[iner], l2_penalty_factor[penalty])
            loss = mlp.train(train_x[batch_start:batch_end,...],train_y[batch_start:batch_end,...],int(learning_rate[lr]), inertia_of_momentum[iner], l2_penalty_factor[penalty])
            
            
            
            
            ####mlp lai cll garda aba kata kata k hunxa network ma layer haru ma kaam gar ani tespaxi directly tesko loss return garne 
            #####actual loss function ma k garne 
            
            
            
            
            
            
            
            
            
            
            # INSERT YOUR CODE FOR EACH MINI_BATCH HERE
			# MAKE SURE TO UPDATE total_loss
            print(
                '\r[Epoch {}, mb {}]    Avg.Loss = {:.3f}'.format(
                    epoch + 1,
                    b + 1,
                    total_loss,
                ),
                end='',
            )
            
            #after each mini bach update you want to update the momentum value
            sys.stdout.flush()
		# INSERT YOUR CODE AFTER ALL MINI_BATCHES HERE
		# MAKE SURE TO COMPUTE train_loss, train_accuracy, test_loss, test_accuracy
        print()
        print('    Train Loss: {:.3f}    Train Acc.: {:.2f}%'.format(
            train_loss,
            100. * train_accuracy,
        ))
        print('    Test Loss:  {:.3f}    Test Acc.:  {:.2f}%'.format(
            test_loss,
            100. * test_accuracy,
        ))


IndentationError: expected an indented block (<ipython-input-7-2d0dbef09b2a>, line 102)